In [1]:
import yaml
import os
import numpy as np
import copy

In [2]:
base_config={
                   #'model_name': ,
                   #'model_path': ,
                   #'device': ,
                   'class_groups': [[2*i,2*i+1] for i in range(5)],
                   'dataset_name':'MNIST' ,
                   #'dataset_type':,
                   #'data_root':,
                   #'xpl_root':,
                   #'coef_root':,
                   #'save_dir': ,
                   'validation_size': 2000,
}


In [3]:
def create_config_local(config, config_name):
    config['data_root'] = '/home/fe/yolcu/Documents/Code/Datasets/'
    config['xpl_root'] = f"/home/fe/yolcu/Documents/Code/THESIS/explanations/{config['dataset_name']}/{config['dataset_type']}/{config['model_name']}_{config['dataset_type']}/{config['xai_method']}"
    if config['xai_method'] in ['mcsvm', 'representer']:
        base_config['coef_root']=base_config['xpl_root']
    elif 'coef_root' in base_config.keys():
        base_config.pop('coef_root')
    config['device'] = 'cuda'
    config['model_path']=f"/home/fe/yolcu/Documents/Code/THESIS/checkpoints/{config['dataset_name']}/{config['dataset_type']}/{config['model_name']}_{config['dataset_type']}/{config['dataset_name']}_{config['model_name']}"
    config['save_dir'] = '/home/fe/yolcu/Documents/Code/THESIS/test_output'
    path = f"local/evaluate/{config['dataset_name']}"
    os.makedirs(path, exist_ok=True)
    
    with open(f"{path}/{config_name}.yaml", 'w') as outfile:
        yaml.dump(config, outfile, default_flow_style=False)
        
def create_config_cluster(config, config_name):
    config['device']='cuda'
    config['xpl_root'] = '/mnt/explanations/'
    if config['xai_method'] in ['mcsvm', 'representer']:
        base_config['coef_root']=base_config['xpl_root']
    elif 'coef_root' in base_config.keys():
        base_config.pop('coef_root')
    config['data_root']='/mnt/dataset'
    config['model_path']=f"/mnt/checkpoints/{config['dataset_name']}/{config['dataset_type']}/{config['model_name']}_{config['dataset_type']}/{config['dataset_name']}_{config['model_name']}"
    config['save_dir'] = '/mnt/outputs/'
    
    path = f"cluster/evaluate/{config['dataset_name']}"
    os.makedirs(path, exist_ok=True)
    
    with open(f"{path}/{config_name}.yaml", 'w') as outfile:
        yaml.dump(config, outfile, default_flow_style=False)

In [4]:
model_names=['basic_conv', 'homo_conv']#,'basic_fc', 'homo_fc']
dataset_types=['std', 'group', 'corrupt', 'mark']
xai_methods=['similarity', 'representer', 'mcsvm', 'influence']#, 'tracin'] 

In [5]:
for name in model_names:
    base_config['model_name']=name       
    for dstype in dataset_types:
        if dstype=='group':
            base_config['num_classes']=len(base_config['class_groups'])
        else:
            base_config['num_classes']=10
        base_config['dataset_type']=dstype
        for xai_method in xai_methods:
            base_config['xai_method']=xai_method
            create_config_cluster(base_config, f"{name}_{dstype}_{xai_method}")
            create_config_local(base_config, f"{name}_{dstype}_{xai_method}")